# 라이브러리 임포트

In [1]:
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

In [2]:
import os

os.mkdir('templates')
os.mkdir('static')

In [3]:
import numpy as np
import math
import torch
import torch.nn.functional as F
import torch.nn as nn
import os
import glob
from PIL import Image
from PIL import JpegImagePlugin
import argparse
import matplotlib.pyplot as plt
import easydict

# model pth 다운로드

In [4]:

import gdown

url = 'https://drive.google.com/u/1/uc?id=1lQ_L-DpNjRFPZAOurTpS-0fCkqlNOLXI'

annotation = 'model.zip'
gdown.download(url,annotation,quiet = False)


Downloading...
From: https://drive.google.com/u/1/uc?id=1lQ_L-DpNjRFPZAOurTpS-0fCkqlNOLXI
To: /content/model.zip
29.2MB [00:00, 56.3MB/s]


'model.zip'

In [5]:
!unzip -qq 'model.zip'

# Dataset 전처리

In [6]:
def read_q_table(file_name):
    jpg = JpegImagePlugin.JpegImageFile(file_name)
    qtable = JpegImagePlugin.convert_dict_qtables(jpg.quantization)
    Y_qtable = qtable[0]
    Y_qtable_2d = np.zeros((8, 8)) 

    qtable_idx = 0
    for i in range(0, 8):
        for j in range(0, 8):
            Y_qtable_2d[i, j] = Y_qtable[qtable_idx]
            qtable_idx = qtable_idx + 1

    return Y_qtable_2d

#caclulate DCT basis
def cal_scale(p,q):
	if p==0:
		ap = 1/(math.sqrt(8))
	else:
		ap = math.sqrt(0.25)
	if q==0:
		aq = 1/(math.sqrt(8))
	else:
		aq = math.sqrt(0.25)

	return ap,aq

def cal_basis(p,q):
	basis = np.zeros((8,8))
	ap,aq = cal_scale(p,q)
	for m in range(0,8):
		for n in range(0,8):
			basis[m,n] = ap*aq*math.cos(math.pi*(2*m+1)*p/16)*math.cos(math.pi*(2*n+1)*q/16)

	return basis

def load_DCT_basis_64(): 
	basis_64 = np.zeros((8,8,64))
	idx = 0
	for i in range(8):
		for j in range(8):
			basis_64[:,:,idx] = cal_basis(i,j)
			idx = idx + 1
	return basis_64

def load_DCT_basis_torch():
    DCT_basis_64 = load_DCT_basis_64()
    np_basis = np.zeros((64, 1, 8, 8)) #outchannel, inchannel, height, width
    for i in range(64):
        np_basis[i,0,:,:] = DCT_basis_64[:,:,i]

    torch_basis = torch.from_numpy(np_basis)
    return torch_basis

def calculate_f1(gt_path, result_array):
  a, b = result_array.shape

  gt_img = Image.open(gt_path).convert("L")
  gt_img = gt_img.resize((b, a)) 
  gt_arr = np.asarray(gt_img)

  threshold = 160
  gt_thres = np.zeros((a, b)).astype('uint8')
  res_thres = np.zeros((a, b)).astype('uint8')

  for i in range(0, a):
    for j in range (0, b):
      if gt_arr[i][j] > threshold:
        gt_thres[i][j] = 255
      if result_array[i][j] > threshold:
        res_thres[i][j] = 255
    
  # calculate f1 score, accuracy
  tp, fp, fn, tn = 0, 0, 0, 0  

  for i in range(0, a):
    for j in range (0, b):
      if gt_thres[i][j] == 255 and res_thres[i][j] == 255:    #tp
        tp = tp+1
      elif gt_thres[i][j] == 255 and res_thres[i][j] == 0:    # fn
        fn = fn+1
      elif gt_thres[i][j] == 0 and res_thres[i][j] == 255:    # fp
        fp = fp+1
      elif gt_thres[i][j] == 0 and res_thres[i][j] == 0:  # tn
        tn = tn+1
  
  f1_score = (2*tp)/((2*tp)+fp+fn)
  accuracy = (tp+tn)/(tp+tn+fp+fn)

  return f1_score, accuracy

def _extract_patches(Y, patch_size, stride):
    patches=list()
    h, w = Y.shape[0:2] # height, width
    
    H = (h - patch_size) // stride
    W = (w - patch_size) // stride

    # patch: one JPEG block
    # patches: whole JPEG blocks
    for i in range(0,H*stride, stride):
        for j in range(0,W*stride,stride):
            patch = Y[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
            
    return patches, H, W
    
def localizing_double_JPEG(Y, qvectors, net):
    net.eval()
    result=0
    PATCH_SIZE = 256

    default_stride = 32 
    default_batchsize = 32 

    
    qvectors = torch.from_numpy(qvectors).float()
    qvectors = qvectors.to(device) 
    qvectors = torch.unsqueeze(qvectors, axis=0)

    patches, H, W = _extract_patches(Y, patch_size=PATCH_SIZE, stride=default_stride)

    # 256 * 256 pathces DJPEG detection
    result = np.zeros((H, W))

    num_batches = math.ceil(len(patches) / default_batchsize)

    result_flatten = np.zeros((H*W))
    for i in range(num_batches):

        # batch_Y: one batch of image patches of Y channel
        if i==(num_batches-1): #last batch
            batch_Y = patches[i*default_batchsize:]
        else:
            batch_Y = patches[i*default_batchsize:(i+1)*default_batchsize]

        batch_size = len(batch_Y) 
        batch_Y = np.array(batch_Y) 

        batch_Y = torch.unsqueeze(torch.from_numpy(batch_Y).float().to(device), axis=1)
        
        batch_qvectors = torch.repeat_interleave(qvectors, batch_size, dim=0)

        batch_output = net(batch_Y, batch_qvectors)
        batch_output = F.softmax(batch_output, dim=1)

        result_flatten[(i*default_batchsize):(i*default_batchsize)+batch_size] = \
                batch_output.detach().cpu().numpy()[:,0]

    result = np.reshape(result_flatten, (H, W))

    return result


# Model 1~4

In [7]:
class Djpegnet1(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, device, n_layers=1):
        super(Djpegnet1, self).__init__()
        self.dct_basis = load_DCT_basis_torch().float()
        self.dct_basis = self.dct_basis.to(device)
 
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=7)
        self.bn1 = nn.BatchNorm1d(num_features = hidden_size)
        self.pool1 = nn.AvgPool1d(kernel_size=2)
 
        self.conv2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=7)
        self.bn2 = nn.BatchNorm1d(num_features = hidden_size)
        self.pool2 = nn.AvgPool1d(kernel_size=2)
 
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, dropout=0.2)
        
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 2)
 
    def forward(self, x, qvectors, hidden=None):
        # feature extraction
        with torch.no_grad(): 
            gamma=1e+06 # 10^6
            x = F.conv2d(x, self.dct_basis, stride=8) 
            for b in range(-80, 81): 
                x_ = torch.sum(torch.sigmoid(gamma*(x-b)), axis=[2,3])/1024 
                x_ = torch.unsqueeze(x_, axis=1) 
                if b==-80:
                    features = x_
                else:
                    features = torch.cat([features, x_], axis=1)
            features = features[:, 0:160, :] - features[:, 1:161, :]
            features = torch.squeeze(features, axis=1)
            features = features[:,:,1:64] # remove DC values
        
        output = torch.relu(self.bn1(self.conv1(features)))
        output = self.pool1(output)
 
        output = torch.relu(self.bn2(self.conv2(output)))
        output = self.pool2(output)
 
        output = output.transpose(1, 2)
 
        output, hidden = self.gru(output, hidden)
 
        output_flat = output[:, -1, :]
        output = torch.cat([qvectors, output_flat], axis=1)
        output = output_flat
 
        output = torch.relu(self.fc1(output))
        output = self.fc2(output)
 
        return output

In [8]:
class Djpegnet2(nn.Module): 
  
    def __init__(self, input_size, hidden_size, output_size, device, n_layers=1):
        super(Djpegnet2, self).__init__()
        self.dct_basis = load_DCT_basis_torch().float()
        self.dct_basis = self.dct_basis.to(device)
 
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=5)
        self.bn1 = nn.BatchNorm1d(num_features = hidden_size)
        self.pool1 = nn.AvgPool1d(kernel_size=2)
 
        self.conv2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=5)
        self.bn2 = nn.BatchNorm1d(num_features = hidden_size)
        self.pool2 = nn.AvgPool1d(kernel_size=2)
 
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, dropout=0.35)
        
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 2)
 
    def forward(self, x, qvectors, hidden = None):
        # feature extraction
        with torch.no_grad(): 
            gamma=1e+06 # 10^6
            x = F.conv2d(x, self.dct_basis, stride=8) 
            for b in range(-60, 61): 
                x_ = torch.sum(torch.sigmoid(gamma*(x-b)), axis=[2,3])/1024 
                x_ = torch.unsqueeze(x_, axis=1) 
                if b==-60:
                    features = x_
                else:
                    features = torch.cat([features, x_], axis=1)
            features = features[:, 0:120, :] - features[:, 1:121, :]
            features = torch.squeeze(features, axis=1)
            features = features[:,:,1:64] # remove DC values
        
        output = torch.relu(self.bn1(self.conv1(features)))
        output = self.pool1(output)
 
        output = torch.relu(self.bn2(self.conv2(output)))
        output = self.pool2(output)
 
        output = output.transpose(1, 2)
 
        output, hidden = self.gru(output, hidden)
 
        output_flat = output[:, -1, :]
        output = torch.cat([qvectors, output_flat], axis=1)
        output = output_flat 
 
        output = torch.relu(self.fc1(output))
        output = self.fc2(output)
 
        return output


In [9]:
class Djpegnet3(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, device, n_layers=1):
        super(Djpegnet3, self).__init__()
        self.dct_basis = load_DCT_basis_torch().float()
        self.dct_basis = self.dct_basis.to(device)
 
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=7)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.pool1 = nn.AvgPool1d(kernel_size=2)
 
        self.conv2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.pool2 = nn.AvgPool1d(kernel_size=2)
 
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, bidirectional=True, dropout=0.2)
        
        self.fc1 = nn.Linear(2*hidden_size, 128)
        self.fc2 = nn.Linear(128, 2)
 
    def forward(self, x, qvectors, hidden=None):
        # feature extraction
        with torch.no_grad(): 
            gamma=1e+06 # 10^6
            x = F.conv2d(x, self.dct_basis, stride=8) 
            for b in range(-60, 61): 
                x_ = torch.sum(torch.sigmoid(gamma*(x-b)), axis=[2,3])/1024 
                x_ = torch.unsqueeze(x_, axis=1) 
                if b==-60:
                    features = x_
                else:
                    features = torch.cat([features, x_], axis=1)
                    
            features = features[:, 0:120, :] - features[:, 1:121, :]
            features = torch.squeeze(features, axis=1)
            features = features[:, :, 1:64]
        
        output = torch.relu(self.conv1(features))
        output = torch.relu(self.bn1(output))
        output = self.pool1(output)
 
        output = torch.relu(self.conv2(output))
        output = torch.relu(self.bn2(output))
        output = self.pool2(output)
 
        output = output.transpose(1, 2)
  
        output, hidden = self.gru(output, hidden)
 
        output_flat = output[:, -1, :]
        output = torch.cat([qvectors, output_flat], axis=1)
        output = output_flat
 
        output = torch.relu(self.fc1(output))
        output = self.fc2(output)
 
        return output

In [10]:
class Djpegnet4(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, device, n_layers=1):
        super(Djpegnet4, self).__init__()
        self.dct_basis = load_DCT_basis_torch().float()
        self.dct_basis = self.dct_basis.to(device)
 
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=5)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.pool1 = nn.AvgPool1d(kernel_size=2)
 
        self.conv2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=5)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.pool2 = nn.AvgPool1d(kernel_size=2)
 
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, bidirectional=True, dropout=0.3)
        
        self.fc1 = nn.Linear(2*hidden_size, 128)
        self.fc2 = nn.Linear(128, 2)
 
    def forward(self, x, qvectors, hidden=None):
        # feature extraction
        with torch.no_grad():
            gamma=1e+06 # 10^6
            x = F.conv2d(x, self.dct_basis, stride=8) 
            for b in range(-60, 61): 
                x_ = torch.sum(torch.sigmoid(gamma*(x-b)), axis=[2,3])/1024 
                x_ = torch.unsqueeze(x_, axis=1) 
                if b==-60:
                    features = x_
                else:
                    features = torch.cat([features, x_], axis=1)
            features = features[:, 0:120, :] - features[:, 1:121, :]
            features = torch.squeeze(features, axis=1)
        
        output = torch.relu(self.conv1(features))
        output = torch.relu(self.bn1(output))
        output = self.pool1(output)
 
        output = torch.relu(self.conv2(output))
        output = torch.relu(self.bn2(output))
        output = self.pool2(output)
 
        output = output.transpose(1, 2)
  
        output, hidden = self.gru(output, hidden)
        
        output_flat = output[:, -1, :]
        output = torch.cat([qvectors, output_flat], axis=1)
        output = output_flat
 
        output = torch.relu(self.fc1(output))
        output = self.fc2(output)

 
        return output

# 모델 삽입

In [11]:
global net1, net2, net3, net4
global device

model_dir_name = './model'

#######################################
selected_model1 = 'net1'
selected_model2 = 'net2'
selected_model3 = 'net3'
selected_model4 = 'net4'

model_pth1 = selected_model1 + '.pth'
model_pth2 = selected_model2 + '.pth'
model_pth3 = selected_model3 + '.pth'
model_pth4 = selected_model4 + '.pth'
#######################################

model_dir_name1 = os.path.join(model_dir_name, model_pth1)
model_dir_name2 = os.path.join(model_dir_name, model_pth2)
model_dir_name3 = os.path.join(model_dir_name, model_pth3)
model_dir_name4 = os.path.join(model_dir_name, model_pth4)

args = easydict.EasyDict({ "input_size": 120, "hidden_size": 256, "output_size": 2, "seq_size": 64, "n_layers": 2})

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
#load pre-trained weights
net1 = Djpegnet1(160, args.hidden_size, args.output_size, device, args.n_layers)
net2 = Djpegnet2(120, args.hidden_size, args.output_size, device, args.n_layers)
net3 = Djpegnet3(args.input_size, args.hidden_size, args.output_size, device, args.n_layers)
net4 = Djpegnet4(args.input_size, args.hidden_size, args.output_size, device, args.n_layers)
    
net1.load_state_dict(torch.load(model_dir_name1))
net2.load_state_dict(torch.load(model_dir_name2))
net3.load_state_dict(torch.load(model_dir_name3))
net4.load_state_dict(torch.load(model_dir_name4))

net1.to(device)
net2.to(device)
net3.to(device)
net4.to(device)

Djpegnet4(
  (conv1): Conv1d(120, 256, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv2): Conv1d(256, 256, kernel_size=(5,), stride=(1,))
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (gru): GRU(256, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

# HTML

In [12]:
text = '''
<html>

<head>
<title>Capstone Image Forensic Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>

    <body>
    <p><h1 align="center">Capstone Image Forensic Website</h1></p>

<div class="container">
<div class="row">
    <h2>Select a file to upload</h2>

     <form action = "/fileUpload" method = "POST" enctype = "multipart/form-data">
        <dl>
            <p>
                <input type="file" name="file" class="form-control" autocomplete="off" required>
            </p>
        </dl>
        <p>
            <input type="submit" value="Submit" class="btn btn-info">
        </p>
    </form>
</div>
</div>

        </form>
    </body>
</html>
'''
file = open("templates/upload.html","w")
file.write(text)
file.close()

In [13]:
text = '''
<html>

<head>
<title>Capstone Image Forensic Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>


    <body>
    <p><h1 align="center">Capstone Image Forensic Website</h1></p>
    <div class="container">
        <h2 align="center"> {{ data }} File Analysis Results </h2>
        <br><br>

  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
Input Image
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
Our Result
</div>
  </div>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src="{{ori_path}}" alt="input_image"  width="250">
{% endif %}
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src={{img_path}} alt="ground_truth" width="250">
{% endif %}
</div>
</div>



</div>
<br><br><br>








<div class="container">
<div class="row">
    <h2>Select a Ground Truth file to upload</h2>

    <form action = "/gtUpload" method = "POST" enctype = "multipart/form-data">
        <dl>
            <p>
                <input type="file" name="gtfile" class="form-control" autocomplete="off" required>
            </p>
        </dl>
        <p>
            <input type="submit" value="Submit" class="btn btn-info">
        </p>
    </form>

</div>
</div>

    </body>
</html>
'''







file = open("templates/showResult.html","w")
file.write(text)
file.close()


In [14]:
text = '''
<html>

<head>
<title>Capstone Image Forensic Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>


    <body>
    <p><h1 align="center">Capstone Image Forensic Website</h1></p>
    <div class="container">
        <h2 align="center">Our Result With Ground-Truth </h2>
        <br><br><br>


<div style=" float: left; padding: 10px; width: 33%;text-align:center">
Input Image
</div>

<div style=" float: left; padding: 10px; width: 33%;text-align:center">
Our Result
</div>

<div style="float: left; padding: 10px; width: 33%;text-align:center">
Ground Truth
</div>
<br>





<div style="float: left; padding: 10px; width: 33%;text-align:center">
{% if True %}
	<img src="{{ori_path}}" alt="input_image"  width="350">
{% endif %}
</div>

<div style=" float: left; padding: 10px; width: 33%;text-align:center">
{% if True %}
	<img src={{img_path}} alt="ground_truth" width="350">
{% endif %}
</div>

<div style=" float: left; padding: 10px; width: 33%;text-align:center">
{% if True %}
	<img src={{gt_path}} alt="result_image" width="350">
{% endif %}
</div>



</div>

    </body>
</html>
'''
file = open("templates/showWithGT.html","w")
file.write(text)
file.close()


# FLASK

In [ ]:
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
app = Flask(__name__)
run_with_ngrok(app)   


@app.route('/fileUpload', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        f.save('static/' + secure_filename(f.filename))
        print('파일 업로드 성공!(static/' ,secure_filename(f.filename) ,')')
        return make_result(secure_filename(f.filename))


# 업로드 HTML 렌더링
@app.route('/')
def render_file():
    return render_template('upload.html')


# GT 파일 업로드 처리
@app.route('/gtUpload', methods = ['GET', 'POST'])
def upload_gt():
    print("in upload_gt")
    if request.method == 'POST':
        f = request.files['gtfile']
        f.save('static/' + secure_filename(f.filename))
        gt_dir_path = 'static/' + secure_filename(f.filename)
        print('GT 파일 업로드 성공!(' ,gt_dir_path , ')')
        return render_template('showWithGT.html', ori_path = file_dir_path, img_path = result_dir_path, gt_path = gt_dir_path)

@app.route('/resultShow')
def make_result(filename):
  global file_dir_path
  global result_dir_path
  file_dir_path = 'static/' + filename
  result_dir_path = 'static/' + 'result_' + filename

  im = Image.open(file_dir_path)
  im = im.convert('YCbCr')
  Y = np.array(im)[:,:,0] # Y channel  
  qvector = read_q_table(file_dir_path).flatten()

  result1 = localizing_double_JPEG(Y, qvector, net1)
  print("result1: 완료")
  result2 = localizing_double_JPEG(Y, qvector, net2)
  print("result2: 완료")
  result3 = localizing_double_JPEG(Y, qvector, net3)
  print("result3: 완료")
  result4 = localizing_double_JPEG(Y, qvector, net4)
  print("result4: 완료")

  result_mean = (result1 + result2 + result3 + result4) / 4

  result = result_mean*255

  
  a, b = result.shape
  threshold = 140
  for i in range(0, a):
    for j in range (0, b):
       if result[i][j] < threshold:
         result[i][j] = 0 

  result = result.astype('uint8')  
  img_result = Image.fromarray(result)
  img_result.convert("L")

  img_result.save(result_dir_path)
  print("이미지 분석 완료")
  return render_template('showResult.html', data = filename, ori_path = file_dir_path, img_path = result_dir_path)



if __name__ == '__main__':
    # 서버 실행
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b00ee52b3322.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Jun/2021 09:40:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 09:40:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Jun/2021 09:40:47] "GET / HTTP/1.1" 200 -
[2021-06-14 09:41:01,139] ERROR in app: Exception on /fileUpload [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [14/Jun/20

파일 업로드 성공!(static/ splicing.jpg )
result1: 완료
result2: 완료
result3: 완료


...

result4: 완료
이미지 분석 완료


127.0.0.1 - - [14/Jun/2021 09:41:33] "GET /static/splicing.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 09:41:38] "GET /static/result_splicing.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 09:41:45] "POST /gtUpload HTTP/1.1" 200 -


in upload_gt
GT 파일 업로드 성공!( static/splicing_gt.jpg )


127.0.0.1 - - [14/Jun/2021 09:41:45] "GET /static/splicing_gt.jpg HTTP/1.1" 200 -
[2021-06-14 09:41:47,052] ERROR in app: Exception on /gtUpload [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [14/Jun/2021 09:41:47] "GET /gtUpload HTTP/1.1" 500 -


in upload_gt
